In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow.keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split

import cv2
import os

import numpy as np

In [13]:
!unzip -q dataset.zip

In [2]:
# Preprocessing

import glob

blocked = glob.glob('dataset/Blocked/*.*')
free = glob.glob('dataset/Free/*.*')

data = []
labels = []

for i in blocked:   
    image = tf.keras.preprocessing.image.load_img(i, color_mode='rgb', target_size = (224,224))
    image = np.array(image)
    data.append(image)
    labels.append(0)
for i in free:   
    image = tf.keras.preprocessing.image.load_img(i, color_mode='rgb', target_size = (224,224))
    image = np.array(image)
    data.append(image)
    labels.append(1)
    
data = np.array(data)
labels = np.array(labels)

In [3]:
print(data.shape)

(141, 224, 224, 3)


In [3]:
# Split dataset 80/20 (x - data, y - labels)

X_train, x_test, Y_train, y_test = train_test_split(data, labels, test_size = 0.2,
                                                random_state = 42)

In [4]:
# Reshape labels as 2d-tensor (the first dim will is the batch dim and the second is the scalar label)

Y_train = np.asarray(Y_train).astype('float32').reshape((-1,1))
y_test = np.asarray(y_test).astype('float32').reshape((-1,1))

print(Y_train.shape)
print(y_test.shape)
print(X_train.shape)
print(x_test.shape) 

(112, 1)
(29, 1)
(112, 224, 224, 3)
(29, 224, 224, 3)


In [6]:
# Make the CNN model

model = Sequential()
model.add(Conv2D(32, 3,padding="same", activation="relu", input_shape=(224,224,3))) # Convolutional layer
model.add(MaxPool2D()) # Pooling layer

model.add(Conv2D(32, 3, padding="same", activation="relu"))
model.add(MaxPool2D())

model.add(Conv2D(64, 3, padding="same", activation="relu"))
model.add(MaxPool2D())
model.add(Dropout(0.4)) # Randomly select 40% of the neurons and set their weights to 0 for one iteration
                        # to prevent overfitting

model.add(Flatten()) # Converts the pooled feature map to a single column 
                     # that is passed to the fully connected layer
    
model.add(Dense(128,activation="relu")) # Fully connected layer
model.add(Dense(2, activation="softmax")) # Output layer

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 28, 28, 64)        0

In [ ]:
# Compile the model (using Adam as optimizer and SparseCategoricalCrossentropy as the loss function)

opt = Adam(lr = 0.001)
model.compile(optimizer = opt, 
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics = ['accuracy'])

# Train model

model_train = model.fit(X_train, Y_train, epochs = 6, validation_data = (x_test, y_test))

In [13]:
# Estimate model

loss, acc = model.evaluate(x_test, y_test, verbose=0)
print(f"test accuracy {acc*100} %")

test accuracy 93.1034505367279 %


In [14]:
# Save model

model.save('best_model.h5')
print('model saved')

model saved


In [7]:
model = tf.keras.models.load_model('best_model.h5')
pred = model.predict(x_test)
print(np.round(pred,2))

[[1.   0.  ]
 [0.   1.  ]
 [1.   0.  ]
 [1.   0.  ]
 [0.   1.  ]
 [0.   1.  ]
 [0.   1.  ]
 [0.   1.  ]
 [0.1  0.9 ]
 [1.   0.  ]
 [1.   0.  ]
 [1.   0.  ]
 [1.   0.  ]
 [0.98 0.02]
 [0.   1.  ]
 [0.   1.  ]
 [1.   0.  ]
 [0.89 0.11]
 [1.   0.  ]
 [0.   1.  ]
 [1.   0.  ]
 [0.   1.  ]
 [0.   1.  ]
 [0.   1.  ]
 [1.   0.  ]
 [1.   0.  ]
 [1.   0.  ]
 [1.   0.  ]
 [0.37 0.63]]
